In [14]:
import torch
import torchvision
import torchvision.datasets as datasets

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print(torch.__version__)

1.0.1.post2


## Load the MNIST dataset

In [15]:
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True,
                                transform=None)

100.1%

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


113.5%

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


100.4%

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz


180.4%

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [16]:
mnist_testset = datasets.MNIST(root='../data', train=False, download=True,
                               transform=None)

In [9]:
len(mnist_trainset), len(mnist_testset)

(60000, 10000)

## Net architecture